In [2]:
using ComponentArrays, Lux, DiffEqFlux, OrdinaryDiffEq, Optimization,
    OptimizationOptimJL, OptimizationOptimisers, Random, Plots

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [8]:
rng = Random.default_rng()
u0 = Float64[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan..., length=datasize)

function tureODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u .^ 3)' * true_A)'
end

prob_trueode = ODEProblem(tureODEfunc, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(),  saveat = tsteps))

2×30 Matrix{Float64}:
 2.0  1.94651   1.74178  1.23834  0.577135  …  1.40964   1.37132   1.28952
 0.0  0.798833  1.46473  1.80877  1.86465      0.466229  0.744667  0.987543

In [11]:
dudt2 = Chain(x -> x .^ 3, Dense(2,50, tanh), Dense(50,2))
p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(), saveat=tsteps)


NeuralODE(
    model = Chain(
        layer_1 = WrappedFunction(#13),
        layer_2 = Dense(2 => 50, tanh_fast),  # 150 parameters
        layer_3 = Dense(50 => 2),       # 102 parameters
    ),
)         # Total: 252 parameters,
          #        plus 0 states.